In [791]:
import folium
import re
import fileinput
import pandas as pd
from folium import IFrame
from folium import plugins

In [792]:
#VEGAS_COORDINATES = (36.1699, -115.1398)
TORONTO_COORDINATES = (43.6532, -79.3832)
#CALGARY_COORDINATES = (51.0486, -114.0708)
map = folium.Map(location=TORONTO_COORDINATES, zoom_start=11)

In [793]:
pos_df = pd.read_csv('datasets/pos_bsns_t_amer.csv')
neg_df = pd.read_csv('datasets/neg_bsns_t_amer.csv')
pos_df = pos_df[pos_df['RestaurantsPriceRange2'] == 1.0]
neg_df = neg_df[neg_df['RestaurantsPriceRange2'] == 1.0]

In [794]:
pos_topics = list(pos_df)[9:15]
neg_topics = list(neg_df)[9:15]

In [795]:
def get_pos_topics(id):
    positive = pos_df.loc[pos_df['business_id'] == id]
    pos_scores=[0, 0, 0, 0, 0, 0]
    if not positive.empty:
        pos_scores = positive.iloc[0,9:15].tolist()
    return pos_scores

def html_text(business_id, stars, neg_scores, name):
    html_topics1 = ['','','','','','']
    html_topics2 = ['','','','','','']
    
    pos_scores = get_pos_topics(business_id)
    for m in range(0, len(pos_topics)):
        html_topics1[m] = '{:18}: {:.2f}%'.format(pos_topics[m], pos_scores[m]*100)
    for n in range(0, len(neg_topics)):
        html_topics2[n] = '{:18} {:.2f}%'.format(neg_topics[n], neg_scores[n]*100)
        
    pos_topics_text = 'Positive Topics<br>' + '<br>'.join(html_topics1)
    neg_topics_text = 'Negative Topics<br>' + '<br>'.join(html_topics2)
    stars_text = '<h3> Stars: ' + '{:.2f}'.format(stars) + '</h3>'
    name_text = '<h3> Name: ' + name + '</h3>'
    return name_text + stars_text + '<p>' + pos_topics_text + '<br><br>' + neg_topics_text + '</p>'

def get_popup(business_id, stars, neg_scores, name):
    html = "" + html_text(business_id, stars, neg_scores, name) + ""
    iframe = folium.IFrame(html=html, width=300, height=200)
    return folium.Popup(iframe, max_width=1000)

In [796]:
def star_color(stars):
    if stars <= 1.5:
        return '#FF0000'
    elif stars <= 2.5:
        return '#FFAD00'
    elif stars <= 3.5:
        return '#FFFF00'
    elif stars < 4.5:
        return '#A1FF00'
    else:
        return '#007200'
    
def get_icon(stars):
    return plugins.BeautifyIcon(border_color='transparent', background_color=star_color(stars), 
                                icon='circle', icon_shape='marker', text_color='#FFF')
    

In [797]:
for row in neg_df.itertuples():
    #print(row)
    probs = [row._10, row._11, row._12, row._13, row._14, row._15]
    folium.Marker(location=[row.latitude, row.longitude], popup=get_popup(row.business_id, row.stars, probs, row.name), 
                  icon=get_icon(row.stars)).add_to(map)
    if row.Index > 600:
        break

In [798]:
import branca.colormap as cm
colormap = cm.LinearColormap(
    ['red', 'orange', 'yellow', 'green'],
    vmin=0, vmax=5
)
colormap.caption = 'Star Rating'
map.add_child(colormap)

In [799]:
map.save('maps/toronto_american_1.html')